
<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI_2024_NLP - Week 1</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: N-gram models </h1>


In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [3]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile

    Returns:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [4]:

print("load training set..")
print("\n")
train_data, vocab = load_data("./train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("\n")
print("load validation set")
valid_data, _ = load_data("./valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107


load validation set


In [5]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count

    Returns:
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab
    # or have a count that is below mincount
    data_with_unk = []
   
    for sentence_list in data:
        for i in range(len(sentence_list)):
            if vocab[sentence_list[i]] < mincount:
                sentence_list[i] = "<unk>"
        data_with_unk.append(sentence_list)

            
    return data_with_unk       
                

In [6]:
print("remove rare words")

train_data = remove_rare_words(train_data, vocab, mincount = 2)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)

#train_data
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [7]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    n (int): size of the n-gram

    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    def _build_ngram(data,n):
        
        counts = defaultdict(lambda: defaultdict(lambda: 0.0))
        for sentence in data:
            sentence = tuple(sentence)
            ##########################################################################
            #                      TODO: Implement this function                     #
            # dict can be indexed by tuples
            # store in the same dict all the ngrams
            # by using the context as a key and the word as a value
            ##########################################################################
    
            total_number_words = 0
            for i in range(len(sentence) - n):
                start = i
                stop = i + n
                context = tuple(sentence[start:stop])
                word = sentence[stop]
                counts[context][word] += 1
            

        prob = defaultdict(lambda: defaultdict(lambda: 0.0))
        # Build the probabilities from the counts
        # Be careful with how you normalize!

        for context in counts.keys():
        # p(w | context) = count(context, w)/ count(context)
            total_count = sum(counts[ context].values())
            for word in counts[context].keys():
                score = (counts[context][word])/total_count
                prob[context][word] = score   
                
        return prob
    
    
    prob = defaultdict(dict) 
    for i in range(1,n+1):
        prob.update(_build_ngram(data, i))  

    return prob

In [8]:
# RUN TO BUILD NGRAM MODEL
n = 10
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)


build ngram model with n =  10


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [9]:
def get_prob(model, context, w):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context

    Retunrs:
    prob(float): probability of this word given the context
    '''

    # code a recursive function over
    # smaller and smaller context
    # to compute the backoff model
    score = None
    context = tuple(context)
    if model.get(context, {}).get(w, 0) != 0:
        score = model[context][w]
    elif len(context) > 0:
        shorter_context = context[:len(context) - 1]
        context = tuple(shorter_context)
        score = get_prob(model, shorter_context, w) 
        if score is not None:
            return score
    else:
        score = 1e-10
    return score



In [10]:
import math
def perplexity(model, data, n):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram

    Retunrs:
    perp(float): the perplexity of the model
    '''
    log_likelihood = 0.0
    num_words = 0
    for sentence in data:
        num_words += len(sentence)
        for i in range(len(sentence)-n):
            start = i 
            stop = i + n
            context = tuple(sentence[start:stop])
            word = sentence[stop]
            
            prob = math.log(get_prob(model, context, word))
            log_likelihood += prob
    log_likelihood *= -1 / num_words
    perplexity = math.exp(log_likelihood)
    return perplexity
   

In [11]:
# COMPUTE PERPLEXITY ON VALIDATION SET
print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 53.34007490670647


In [12]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and
    thier probabilites

    Retunrs:
    words_and_probs(dic): {word: probability of word given context}

    '''
    # code a recursive function over context
    # to find the longest available ngram

    
    words_and_probs = defaultdict(lambda: 0.0)
    context = tuple(context)

    if context in model:
        total_count = sum(model[context].values())
        for word, count in model[context].items():
            probability = count / total_count
            words_and_probs[word] = probability
    else:
        if len(context) > 0:
            shorter_context = context[:len(context) - 1]
            shorter_words_and_probs = get_proba_distrib(model, shorter_context)
            words_and_probs.update(shorter_words_and_probs)
    return words_and_probs  


In [13]:
def generate(model):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }

    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model.


    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    n =10
    #print (model[("<s>")])
    #print (len(model[tuple(sentence)].values()))
    while sentence[-1] != "</s>" and len(sentence)< n:
        
        words_and_probs = get_proba_distrib(model,sentence)
        x = list(words_and_probs.keys())
        y = list(words_and_probs.values())
        
        word_pred = np.random.choice(x, 1, p = y)
        sentence.append(word_pred[0])
    return sentence
        

In [14]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'did', 'you', 'get', 'the', 'money', 'i', 'sent', 'you?', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [20]:
print("load training set 2..")
print("\n")
train_data2, vocab = load_data("./train2.txt")
print(train_data2[0])
print("\n")
print("how :",vocab['how'])
print("load validation set 2")
valid_data2, _ = load_data("./valid2.txt")


load training set 2..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


how : 3195
load validation set 2


In [21]:
n = 3

In [22]:
model = build_ngram(train_data2, n)

In [23]:
perplexity(model,valid_data2, n)

311.2889726403254

In [25]:
generate(model)

['<s>', 'the', 'plane', 'will', 'gets', 'took', 'was', 'we', 'is', 'flew']